In [1]:
!export CUDA_VISIBLE_DEVICES=0

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))
import pdb

<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_1295860/3738920918.py:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if label is '':


In [6]:
emb_hyp = {
    'audio/sample_rate': 8000,
    'embedder/n_fft': 512,
    'embedder/hop': 160,
    'embedder/n_mels': 40,
    'embedder/num_layers': 3,
    'embedder/lstm_hidden': 768,
    'embedder/emb_dim': 256,
    'embedder/pretrained_path': '/mnt/data/zhepei/outputs/tse_semi/spid/vfpt_spid/2021-08-29-22-19-31_blstm_softmax_8k_bs46_4sec/chkpt/chkpt_100000.pt',
    'embedder/trainable': False,
}

from spid_modules.embedder import Embedder
embedder = Embedder(emb_hyp)
embedder.eval()

Loading embedder checkpoint from /mnt/data/zhepei/outputs/tse_semi/spid/vfpt_spid/2021-08-29-22-19-31_blstm_softmax_8k_bs46_4sec/chkpt/chkpt_100000.pt


Embedder(
  (mel_transform): Wav_to_Mel_VF(
    (stft): STFT()
    (mel_transform): Linear(in_features=257, out_features=40, bias=False)
  )
  (lstm): LSTM(40, 768, num_layers=3, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=1536, out_features=256, bias=True)
)

In [26]:
hyp = {
    'dataloader_opts': {
        'batch_size': 1,
        'num_workers': 0,
    },
    'valid_data': '/mnt/data/Speech/wsj_tse/mix_2_spk_cv/meta.csv',
    'base_folder_dm': '/mnt/data/wsj0.8k/si_tr_s/',
    'base_folder_dm_ext': 'wav',
    'sample_rate': 8000,
    'training_signal_len': 24000,
}

In [27]:
from data.data_mixing import static_mixing_prep
import torch
import numpy as np
from speechbrain.dataio.batch import PaddedBatch

# Test static mixing 

In [28]:
ds = static_mixing_prep(hyp, 'valid')
dl = torch.utils.data.DataLoader(
        ds,
        batch_size=hyp["dataloader_opts"]["batch_size"],
        num_workers=hyp["dataloader_opts"]["num_workers"],
        collate_fn=PaddedBatch,
        worker_init_fn=lambda x: np.random.seed(
            int.from_bytes(os.urandom(4), "little") + x
        ),
    )

In [29]:
def listen_batch(batch):
    mix = batch['mix_sig'].data
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    enr = batch['enr_sig'].data
    diff = mix - s1 - s2
    print(abs(mix).max())
    print((diff**2).mean())
    sound(mix[0].numpy(), rate=hyp['sample_rate'], label='mix')
    sound(s1[0].numpy(), rate=hyp['sample_rate'], label='s1')
    sound(s2[0].numpy(), rate=hyp['sample_rate'], label='s2')
    sound(enr[0].numpy(), rate=hyp['sample_rate'], label='enr')

In [30]:
def compute_cos_sim(v1, v2):
    return (v1 * v2).sum() / (torch.norm(v1) * torch.norm(v2))

def check_emb(batch):
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    enr = batch['enr_sig'].data
    s1_emb = embedder(s1)
    s2_emb = embedder(s2)
    enr_emb = embedder(enr)
    pos_sim = compute_cos_sim(s1_emb[0], enr_emb[0]).item()
    neg_sim = compute_cos_sim(s2_emb[0], enr_emb[0]).item()
    print('Positive sim: {} -- Negative sim: {} -- diff: {}'.format(pos_sim, neg_sim, pos_sim-neg_sim))

In [31]:
for i, batch in enumerate(dl):  
    check_emb(batch)
    if i == 20:
        break

Positive sim: 0.578312873840332 -- Negative sim: 0.24754075706005096 -- diff: 0.33077211678028107
Positive sim: 0.7316041588783264 -- Negative sim: 0.16863436996936798 -- diff: 0.5629697889089584
Positive sim: 0.9237152338027954 -- Negative sim: 0.3536315858364105 -- diff: 0.5700836479663849
Positive sim: 0.6988825798034668 -- Negative sim: 0.16954517364501953 -- diff: 0.5293374061584473
Positive sim: 0.7269286513328552 -- Negative sim: 0.5586882829666138 -- diff: 0.16824036836624146
Positive sim: 0.7467442154884338 -- Negative sim: 0.2234179675579071 -- diff: 0.5233262479305267
Positive sim: 0.8882954716682434 -- Negative sim: 0.4721169173717499 -- diff: 0.41617855429649353
Positive sim: 0.5537095665931702 -- Negative sim: 0.1813872754573822 -- diff: 0.37232229113578796
Positive sim: 0.7510630488395691 -- Negative sim: 0.06278032809495926 -- diff: 0.6882827207446098
Positive sim: 0.7496092915534973 -- Negative sim: 0.2045174539089203 -- diff: 0.545091837644577
Positive sim: 0.76424390

In [33]:
for i, batch in enumerate(dl):  
    if i == 1:
        listen_batch(batch)
        break

tensor(0.8990)
tensor(0.)


mix,Your browser does not support the audio element.


s1,Your browser does not support the audio element.


s2,Your browser does not support the audio element.


enr,Your browser does not support the audio element.
